## この章で行うこと

In [1]:
# ライブラリのimport
import numpy as np
import pandas as pd
# 表示桁数の指定
%precision 3
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()

# ライブラリのimport
import scipy as sp
from scipy import stats
import statsmodels.formula.api as smf
import statsmodels.api as sm
# 表示桁数の指定
%precision 3
# グラフをjupyter notebook内に表示
%matplotlib inline

In [2]:
fish = pd.read_csv('3-7-1-fish_length.csv')['length']

In [3]:
fish

0    4.352982
1    3.735304
2    5.944617
3    3.798326
4    4.087688
5    5.265985
6    3.272614
7    3.526691
8    4.150083
9    3.736104
Name: length, dtype: float64

### 点推定

In [5]:
mu = sp.mean(fish)
mu

4.187

標本平均が4.187なので、母平均も4.187と見積もるのが点推定

母分散の点推定値として、普遍分散が用いられる。<br>

In [16]:
sigma_2 = sp.var(fish, ddof = 1)
sigma_2

0.680

### 区間推定

区間推定とは、推定値に幅を持たせた推定方法。<br>

### 信頼係数・信頼区間

信頼係数 : 95%, 99%<br>
信頼区間 : 信頼係数を満たす区間<br>

### 信頼限界

下側信頼限界, 上側信頼限界とも呼ばれる。<br>

### 区間推定

区間推定に必要となるものは以下の３つ<br>
自由度, 標本平均, 標準誤差<br>

In [8]:
# 自由度
df = len(fish) - 1
df

9

In [20]:
sigma = sp.sqrt(sigma_2)
se = sp.sqrt(sigma_2) / sp.sqrt(len(fish))
se

0.261

In [19]:
interval = stats.t.interval(
    alpha = 0.95, df = df, loc = mu, scale = se
)
interval

(3.597, 4.777)

### 信頼区間の幅を決める要素

標本における分散が大きければ「データが平均値から離れている→平均値をあまり信用できない<br>
したがって、信頼区間の幅が広くなる。<br>

In [21]:
se2 = (sigma*10) / sp.sqrt(len(fish))
stats.t.interval(
    alpha = 0.95, df = df, loc = mu, scale=se2
)

(-1.713, 10.087)

サンプルサイズを増やしてみる<br>

In [24]:
df2 = (len(fish)*10)-1
se3 = sigma / sp.sqrt(len(fish) * 10)

In [26]:
stats.t.interval(
    alpha = 0.95, df = df2, loc = mu, scale=se3
)

(4.023, 4.351)

### 信頼区間の求め方の詳細

In [35]:
# 97.5%点を求める
t_975 = stats.t.ppf(q = 0.975, df= df)
t_975

2.262

In [38]:
# 下側信頼限界
lower = mu - t_975 * se
lower

3.772

In [39]:
upper = mu + t_975 * se
upper

4.496

### 区間推定の結果の解釈

信頼係数95%の「95%」は以下のように得られる。<br>

1. 真の母集団分布から標本を抽出s<br>
2. 今回と同じやり方で95%信頼区間を計算<br>
3. この試行をたくさん繰り返す<br>
4. すべての試行のうち、「真の母数」が信頼区間に含まれている割合が95%<br>

In [28]:
be_included_array = np.zeros(20000, dtype='bool')
be_included_array

array([False, False, False, ..., False, False, False])

In [30]:
# 「95%信頼区間を求める」試行を20000回繰り返す
# 信頼区間が母平均（４）を含んでいればTrue
np.random.seed(1)
norm_dist = stats.norm(loc=4, scale=0.8)
for i in range(0, 20000):
    sample = norm_dist.rvs(size=10) # サンプルから10個取り出す
    df = len(sample) - 1 # 自由度
    mu = sp.mean(sample) # 平均
    std = sp.std(sample, ddof = 1) # 
    se = std / sp.sqrt(len(sample))
    interval = stats.t.interval(0.95, df, mu, se)
    if(interval[0] <= 4 and interval[1] >=4):
        be_included_array[i] = True

In [31]:
sum(be_included_array) / len(be_included_array)

0.948